

# Import library

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from PIL import Image


print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Persiapan Dataset

In [ ]:
base_pathDrive = './drive/MyDrive'

# Extract the archive
zip_ref = zipfile.ZipFile(os.path.join(base_pathDrive, 'dataset.zip'), 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Assign training and validation set directories
base_dir = 'tmp/dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat pictures
train_pass_dir = os.path.join(train_dir, 'pass')

# Directory with training dog pictures
train_decline_dir = os.path.join(train_dir, 'decline')

# Directory with validation cat pictures
validation_pass_dir = os.path.join(validation_dir, 'pass')

# Directory with validation dog pictures
validation_decline_dir = os.path.join(validation_dir, 'decline')

Use ImageDataGenerator to get image from directory and using directory name as label

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Found 1400 images belonging to 2 classes.
Found 377 images belonging to 2 classes.


# Training Data

Load Model Mobile Net to transfer learning

In [ ]:
MODULE_HANDLE = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5'
module = hub.load(MODULE_HANDLE)

Retrain and make model layer

In [ ]:
IMAGE_SIZE = (224, 224)

model = tf.keras.Sequential([
        hub.KerasLayer(MODULE_HANDLE,
                       input_shape=IMAGE_SIZE + (3,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              4226432   
                                                                 
 dense_5 (Dense)             (None, 512)               655872    
                                                                 
 dense_6 (Dense)             (None, 1)                 513       
                                                                 
Total params: 4,882,817
Trainable params: 656,385
Non-trainable params: 4,226,432
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

EPOCHS = 20

history = model.fit(train_generator, epochs=EPOCHS, validation_data=validation_generator,verbose=1, steps_per_epoch=70, validation_steps=19)


Epoch 1/20
70/70 [==============================] - 153s 2s/step - loss: 0.5821 - acc: 0.7164 - val_loss: 0.4485 - val_acc: 0.7931
Epoch 2/20
70/70 [==============================] - 145s 2s/step - loss: 0.4475 - acc: 0.7871 - val_loss: 0.4081 - val_acc: 0.8276
Epoch 3/20
70/70 [==============================] - 144s 2s/step - loss: 0.4088 - acc: 0.8043 - val_loss: 0.3901 - val_acc: 0.8143
Epoch 4/20
70/70 [==============================] - 144s 2s/step - loss: 0.3826 - acc: 0.8129 - val_loss: 0.4898 - val_acc: 0.7507
Epoch 5/20
70/70 [==============================] - 144s 2s/step - loss: 0.3386 - acc: 0.8386 - val_loss: 0.4653 - val_acc: 0.8329
Epoch 6/20
70/70 [==============================] - 144s 2s/step - loss: 0.3267 - acc: 0.8514 - val_loss: 0.4908 - val_acc: 0.8064
Epoch 7/20
70/70 [==============================] - 142s 2s/step - loss: 0.2932 - acc: 0.8743 - val_loss: 0.5376 - val_acc: 0.7613
Epoch 8/20
70/70 [==============================] - 141s 2s/step - loss: 0.2782 - a

Saving model

In [ ]:
saved_model_path = './second_model'
model.save(saved_model_path)

KeyboardInterrupt: ignored

In [ ]:
import shutil

shutil.make_archive('second_model', 'zip', './second_model')

'/content/second_model.zip'

# Test Prediksi Model

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(224, 224))
  
  x=image.img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0.5:
    print(fn + " = Pass")
  else:
    print(fn + " = Decline")
 

Saving kaya 3.jpeg to kaya 3.jpeg
Saving kaya 2.jpg to kaya 2.jpg
Saving miskin 3.jpg to miskin 3.jpg
Saving miskin 2.jpg to miskin 2.jpg
Saving miskin 1.jpg to miskin 1.jpg
Saving kaya 1.jpg to kaya 1.jpg
[0.41588128]
kaya 3.jpeg = Decline
[0.00620997]
kaya 2.jpg = Decline
[0.9986228]
miskin 3.jpg = Pass
[0.9999099]
miskin 2.jpg = Pass
[0.9999999]
miskin 1.jpg = Pass
[0.00213863]
kaya 1.jpg = Decline
